In [3]:
#1. get xmlboxes
#2. get xmlboxes img region from bkg
#3. put the img region to new image
import os, datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path

time_str = (datetime.datetime.now()).strftime("%Y%m%d")
generated_image_dir = '/data/darknet/python/' + time_str + '_images_replaced_ordinary_obj_with_bkg/'

xml_dir = '/data/darknet/python/' + time_str + '_xml/'
image_dir = '/data/darknet/python/' + time_str + '_image/'

os.system('mkdir -p ' + xml_dir)
os.system('mkdir -p ' + image_dir)
os.system('mkdir -p ' + generated_image_dir)
background_dir = '/data/darknet/python/' + time_str + '_background/'

SEED_XML_DIR = xml_dir
SEED_IMG_DIR = image_dir
SEED_BKG_DIR = background_dir
GENE_IMG_DIR = generated_image_dir


In [4]:
def past_to_background_from_image_file(file, bboxes, background_img_array, save_name, extend_spaces=0):
    
    img = cv2.imread(file)
    #img = img - 50
    if(img.shape != background_img_array.shape):
        print('shape not match')
        return
    #print(img.shape)
    #print(img)
    img_objs = []
    for bbox in bboxes:
        img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
        img_objs.append(img_obj)
    i = 0
    for bbox in bboxes:
        background_img_array[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])] = img_objs[i]
        i = i+1
    time_mark = datetime.datetime.now()
    time_str = time_mark.strftime("%Y%m%d%H%M%S.%f")
    cv2.imwrite(GENE_IMG_DIR + time_str + '_' + save_name.split('/')[-1], background_img_array)
    return 
def get_obj_from_xml(xml_name):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    return [obj for obj in root.iter('object')]
def get_bboxes_from_etree(etree):
    root = tree.getroot()  
    objects = root.findall('object')
    bboxes = []
    for obj in objects:
        '''
        difficult = obj.find('difficult').text
        cls_ = obj.find('name').text
        if cls_ not in classes_ or int(difficult)==1:
            continue
        '''
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        bboxes.append(b)
    return bboxes
def generate_new_xmlobj(xmlobj_old, new_position, new_size):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text

    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text

    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(new_position[0])
    tag_ymin.text = str(new_position[1])
    tag_xmax.text = str(new_position[0] + new_size[1])
    tag_ymax.text = str(new_position[1] + new_size[0])
    return element_object
    
def past_and_insert(img_obj, img_array, new_position, obj_element, etree):
    new_xmlobj = generate_new_xmlobj(obj_element, new_position, img_obj.shape[:2])
    new_xml_etree = insert_to_xml(new_xmlobj, etree)
    new_pil_img = past_obj_to_background(img_obj, img_array, new_position)
    return new_pil_img, new_xml_etree
    

def insert_to_xml(xml_obj, xml_etree):
    root = xml_etree.getroot()
    root.append(xml_obj)
    return xml_etree
    
def past_obj_to_background(img_obj, img_array, position = (200, 200)):
    img = Image.fromarray(img_array)
    img_obj = Image.fromarray(img_obj)
    img.paste(img_obj, position)
    return img
    
def get_cls_from_xmlobj(obj_element):    
    return obj_element.find('name').text

def get_bbox_from_xmlobj(obj_element):
    xmlbox = obj_element.find('bndbox')
    return [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), 
            float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
def generate_new_position(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    #print(new_position)
    return new_position
def generate_new_bbox(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    new_bbox = [random_x, random_x + img_obj_size[1], random_y, random_y + img_obj_size[0]]
    #print(new_bbox)
    return new_bbox
    
    
def inset_obj_to_an_image_and_xml(img_obj, img, obj_element, etree):
    new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    bboxes = get_bboxes_from_etree(etree)
    while(not check_bbox(new_bbox, bboxes)):
        print('new_bbox not suitable, retry...')
        new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    #print('new_bbox succussful')
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, (new_bbox[0], new_bbox[2]), obj_element, tree)
    
    return new_pil_img, new_xml_etree

def check_bbox(new_bbox, bboxes):
    for bbox in bboxes:
        if(IOU(new_bbox, bbox) > 0.005):
            return False
        continue
    return True

seed_xml_names = os.listdir(SEED_XML_DIR)
seed_bkg_names = os.listdir(SEED_BKG_DIR)
seed_bkg_names.sort()
seed_xml_names.sort()

for xml_name in seed_xml_names:
    print(xml_name)       
    '''if(not ((int(xml_name[:-4]) >= 460500) and int(xml_name[:-4]) < 461000)):
        continue'''
    background_img_data = cv2.imread(SEED_IMG_DIR + xml_name[:-3] + 'jpg')
    if(not xml_name.endswith('.xml')):
        continue
    in_file = open(SEED_XML_DIR + xml_name)
    tree=ET.parse(in_file)
    bboxes = get_bboxes_from_etree(tree)
    past_to_background_from_image_file(SEED_BKG_DIR + seed_bkg_names[0], bboxes, background_img_data, 
                                       SEED_IMG_DIR + xml_name[:-3] + 'jpg')

00006002300.xml
00006002301.xml
00006002302.xml
00006002303.xml
00006002304.xml
00006002305.xml
00006002306.xml
00006002307.xml
00006002308.xml
00006002309.xml
00006002310.xml
00006002311.xml
00006002312.xml
00006002313.xml
00006002314.xml
00006002315.xml
00006002316.xml
00006002317.xml
00006002318.xml
00006002319.xml
00006002320.xml
00006002321.xml
00006002322.xml
00006002323.xml
00006002324.xml
00006002325.xml
00006002326.xml
00006002327.xml
00006002328.xml
00006002329.xml
00006002330.xml
00006002331.xml
00006002332.xml
00006002333.xml
00006002334.xml
00006002335.xml
00006002336.xml
00006002337.xml
00006002338.xml
00006002339.xml
00006002340.xml
00006002341.xml
00006002342.xml
00006002343.xml
00006002344.xml
00006002345.xml
00006002346.xml
00006002347.xml
00006002348.xml
00006002349.xml
00006002350.xml
00006002351.xml
00006002352.xml
00006002353.xml
00006002354.xml
00006002355.xml
00006002356.xml
00006002357.xml
00006002358.xml
00006002359.xml
00006002360.xml
00006002361.xml
00006002

00006002814.xml
00006002815.xml
00006002816.xml
00006002817.xml
00006002818.xml
00006002819.xml
00006002820.xml
00006002821.xml
00006002822.xml
00006002823.xml
00006002824.xml
00006002825.xml
00006002826.xml
00006002827.xml
00006002828.xml
00006002829.xml
00006002830.xml
00006002831.xml
00006002832.xml
00006002833.xml
00006002834.xml
00006002835.xml
00006002836.xml
00006002837.xml
00006002838.xml
00006002839.xml
00006002840.xml
00006002841.xml
00006002842.xml
00006002843.xml
00006002844.xml
00006002845.xml
00006002846.xml
00006002847.xml
00006002848.xml
00006002849.xml
00006002850.xml
00006002851.xml
00006002852.xml
00006002853.xml
00006002854.xml
00006002855.xml
00006002856.xml
00006002857.xml
00006002858.xml
00006002859.xml
00006002860.xml
00006002861.xml
00006002862.xml
00006002863.xml
00006002864.xml
00006002865.xml
00006002866.xml
00006002867.xml
00006002868.xml
00006002869.xml
00006002870.xml
00006002871.xml
00006002872.xml
00006002873.xml
00006002874.xml
00006002875.xml
00006002